In [1]:
## Importing libraries

import pandas as pd
import numpy as np
import sys; sys.path.insert(1, '/Users/EvanCallaghan/Documents/Golf/StrokesGained/GOLF/GlobalScripts')
pd.set_option('display.max_rows', None, 'display.max_columns', None)
import sg_script as sg
import sg_script_editor as sg_editor
import sg_script_multiple_rounds as sg_multiple

In [2]:
## Reading the data

baselines = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Baselines.xlsx')
courses = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Courses.xlsx')
golfers = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Golfers.xlsx')
rounds = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Rounds.xlsx')
shots = pd.read_excel('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Database/Shots.xlsx')

In [ ]:
## Subsetting the data for the correct golfer and round
golfer = golfers[golfers['Name'] == 'John David Coffin'].reset_index(drop = True)
rd = rounds[rounds['roundID'] == 1075].reset_index(drop = True)
shot = shots[shots['roundID'] == 1075].reset_index(drop = True)
course = courses[(courses['courseID'] == rd.at[0, 'courseID']) & (courses['Tees'] == rd.at[0, 'Tees'])].reset_index(drop = True)

## Creating complete data-frame for analysis
shotLevel, roundLevel = sg.initialize(shot, rd, course, baselines)

## Exporting data-frames for analysis:
shotLevel.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/shotLevel.csv', index = False)
roundLevel.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/roundLevel.csv', index = False)
course.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/courseLevel.csv', index = False)

### Multiple round analysis

In [3]:
## Subsetting the data for the correct golfer and rounds
golfer = golfers[golfers['Name'] == 'John David Coffin'].reset_index(drop = True)
rds = rounds[(rounds['Name'] == 'John David Coffin') & (rounds['roundID'] >= 1010)].reset_index(drop = True)
shot = shots[np.isin(shots['roundID'], rds['roundID'].unique())].reset_index(drop = True)

rds['courseTeeCombo'] = rds['Course'] + rds['Tees']
courses['courseTeeCombo'] = courses['Course'] + courses['Tees']
course = courses[np.isin(courses['courseTeeCombo'], rds['courseTeeCombo'].unique())].reset_index(drop = True)

## Creating complete data-frame for analysis
shotLevel, roundLevel = sg_multiple.initialize(shot, rds, course, baselines)

# ## Exporting data-frames for analysis:
# shotLevel.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/shotLevelMultiple.csv', index = False)
# roundLevel.to_csv('/Users/EvanCallaghan/Documents/Golf/StrokesGained/Reporting/roundLevelMultiple.csv', index = False)

In [5]:
trend = roundLevel.groupby(['Date'])[['SG: Total', 'OTT', 'APP', 'ATG', 'PUTT', 'tournamentRound']].mean().reset_index(drop = False)

trend.to_csv('/Users/EvanCallaghan/Desktop/JDrounds.csv', index = False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.style.use('seaborn-darkgrid')
plt.figure(figsize = (16, 8))
palette = plt.get_cmap('Set1')

num = 0
for column in trend.drop('Date', axis = 1):
    num += 1
    plt.plot(trend['Date'], trend[column], marker = '', color = palette(num), linewidth = 1, alpha = 0.9, label = column)

# Add legend
plt.legend(loc = 2, ncol = 2)
 
# Add titles
plt.title("Time-Series Visualization of Strokes Gained Data", loc = 'left', fontsize = 12, fontweight = 0, color = 'orange')
plt.xlabel("Date")
plt.ylabel("Strokes Gained Value")
plt.show()